In [1]:
from datasets  import load_metric
from bleurt import score
from parent import parent
import json
import sys
sys.path.append('../src')
sys.path.append('../TrainedNarrators/')
from data_utils import *

scorer = score.LengthBatchingBleurtScorer('../../../bleurt/BLEURT-20/')
metric_bert = load_metric('bertscore')
metric_sbleu = load_metric('sacrebleu')
metric_meteor = load_metric('meteor')
#metric_bleurt = load_metric('bleurt', 'bleurt-large-512')
metric_rouge = load_metric('rouge')
metric_wer = load_metric('google_bleu')


INFO:tensorflow:Reading checkpoint ../../../bleurt/BLEURT-20/.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: ../../../bleurt/BLEURT-20/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


[nltk_data] Downloading package wordnet to /home/essel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
test_data = json.load(open('../dataset/test set.json'))
test_sample = []
eval_tables = []
for pc in test_data:
    test_sample.append(processInputTableAndNarrations(
        pc, identical_metrics=identicals))
    # eval_tables.append(parseTableStructureForEval(pc,identicals))
rtest_sample = []
reval_tables = []
for pc in test_data:
    rtest_sample.append(processInputTableAndNarrations(
        pc, identical_metrics=identicals))

refs = [[t['narration'].lower() for t in test_sample]]
orf = [[normalize_text(r) for r in refs[0]]]
reff = [normalize_text(s.strip()).split() for s in refs[0]]
trefs = [[normalize_text(r)] for r in refs[0]]
from sacrebleu.tokenizers import tokenizer_13a
def normalize_text(s):
    tokenize_fn = lambda x: tokenizer_13a.Tokenizer13a()(x)
    return tokenize_fn(s.strip().lower())
# Prepare the table for PARENT score
def ParentisePreamble(pc):
    mets = []
    for m,v in zip(pc['metrics'],pc['values']):
        stg = [f'{m}',normalize_text(v).split()]
        mets.append(stg)
    random.shuffle(mets)
    class_labels = pc['classes']
    classes_string = 'or '.join(class_labels[:-1])+' or '+class_labels[-1]
    classes_string_1 = ', '.join(class_labels[:-1])+' and '+class_labels[-1]
    mets.append(['labels',normalize_text(classes_string).split()])
    mets.append(['classes',normalize_text(classes_string_1).split()])
    ds= pc['dataset_attribute'][0]
    if ds =='is_balanced':
        ds = 'balance'
    else:
        ds= 'imbalance'
    #mets.append(['dist',normalize_text(ds.replace('is_','')).strip().split()])
    if len(class_labels)<3:
        mets.append(['mode1',normalize_text('binary').split()])
        #mets.append(['mode2',normalize_text('two-way').split()])
    else:
        if len(class_labels)<3:
            mets.append(['mode1',normalize_text('multi-class').split()])
            #mets.append(['mode2',normalize_text('four-way' if len(class_labels==4) else 'three-way').split()])
    return mets
# Compute the table representations for PARENT metric
typ=table_rep=[ParentisePreamble(pc) for pc in test_sample]

In [5]:
len(reff)

100

In [8]:
def computeParentScore(data,table_rep):
    predic= [normalize_text(s.strip()).split() for s in data]
    #print(len(predic))
    precision, recall, f_score = parent(
    predic,
    reff,
    table_rep,lambda_weight=.8,
    avg_results=True,
        smoothing=.00001,
    n_jobs=32,
    use_tqdm=False)
    #print(precision, recall, f_score)
    return precision, recall, f_score


def computeMeanParentBS(data,table_rep=table_rep,bs=8):
    pre,rec,fs= [],[],[]
    for tt in data:
        #print(len(tt[f'{bs}']))
        #print('ttg ',len(table_rep))
        precision, recall, f_score = computeParentScore(tt[f'{bs}'],table_rep)
        pre.append(precision)
        rec.append(recall)
        fs.append(f_score)
    results = {'precision':roundN(np.mean(pre)*100,2), 
               'recall': roundN(np.mean(rec)*100,2), 'f1': roundN(np.mean(fs)*100,2)}
    return results
def computeMeanBLEURT(data,bs=8):
    b_res = []
    for tt in data:
        gen= [t for t in tt[f'{bs}']]
        score=np.mean(scorer.score(references=refs[0], candidates=gen))
        b_res.append(score)
    return np.mean(b_res)
def computeBERTBLEU(data,bs=8):
    b_res = []
    for tt in data:
        gen= [t for t in tt[f'{bs}']]
        ss= metric_bert.compute(predictions=[gen],references=[refs],lang='en')
        score=ss['f1']
        b_res.append(score)
    return np.mean(b_res)
def computeAutoMetrics(data,bs=8):
    b_res = []
    m_res = []
    r_res = []
    r2_res = []
    r1_res = []
    bleurt_score = computeMeanBLEURT(data,bs)
    for tt in data:
        gen= [normalize_text(t) for t in tt[f'{bs}']]
        score=metric_sbleu.compute(predictions=[gen],references=[orf])
        b_res.append(score['score'])
        
        
        # Meteor
        score_=metric_meteor.compute(predictions=[gen],references=[orf])
        m_res.append(score_['meteor'])
        
        
        # ROUGE
        score=metric_rouge.compute(predictions=[gen],references=orf,use_stemmer=True)
        score= {key: value.high.fmeasure * 100 for key, value in score.items()}
        r_res.append(score['rougeL'])
        r2_res.append(score['rouge2'])
        r1_res.append(score['rouge1'])
        #r_res.append(0)
        
        
    return {'BLEURT':bleurt_score,
            'BLEU': np.mean(b_res),
            'METEOR':np.mean(m_res)*100,
            'rouge1': np.mean(r1_res),
            'rouge2': np.mean(r2_res),
            'rougeL': np.mean(r_res)}
    
def computeEvaluationScores(data,bs=8):
    # Compute the Parent score
    parent_scores = computeMeanParentBS(data,bs=bs)
    # Compute BLEU and METEOR score
    parent_scores.update(computeAutoMetrics(data,bs=bs))  
    return parent_scores

In [9]:
# Load the generated texts
import copy
import json
#t5_small_43,t5_small_128,t5_small_456,t5_small_3087,t5_small_1984
def loadData(main_path,model_base,seeds =[43,128,456,3087,1984]):
    #'../TrainedNarrators/P-NarrationsModels/baselineoutputs_new/t5-small/43/narrations_outputoe.json'
    outputs = {}
    for model in model_base:
        p= main_path+f'/{model}/'
        elems = []
        for s in seeds:
            z= p+f'{s}/narrations_outputoe.json'
            d= json.load(open(z,'rb'))
            elems.append(d)
        outputs[model]= elems
    return outputs

In [10]:
t5_baseline=loadData('../TrainedNarrators/P-NarrationsModels/baselineoutputs_new/',['t5-small','t5-base','t5-large'],)
t5_ef=loadData('../TrainedNarrators/P-NarrationsModels/earlyfusionoutputs_new/',['t5-small','t5-base','t5-large'],)

In [11]:
bart_ef = loadData('../TrainedNarrators/P-NarrationsModels/earlyfusionoutputs_new/',['bart-base','bart-large'],seeds =[48,128,456,3087,1984])
bart_baseline = loadData('../TrainedNarrators/P-NarrationsModels/baselineoutputs_new/',['bart-base','bart-large'],seeds =[48,128,456,3087,1984])

In [12]:
def computeAllScores(pack,bs=8):
    results = {}
    for k,v in pack.items():
        print(k)
        scores = computeEvaluationScores(v,bs=bs)
        results[k] = scores
    return results

In [13]:
t5_scores=computeAllScores(t5_baseline)

t5-small
INFO:tensorflow:Average batch sequence length: 217.28571428571428
INFO:tensorflow:Average batch sequence length: 209.57142857142858
INFO:tensorflow:Average batch sequence length: 210.42857142857142
INFO:tensorflow:Average batch sequence length: 216.57142857142858
INFO:tensorflow:Average batch sequence length: 226.57142857142858
t5-base
INFO:tensorflow:Average batch sequence length: 236.14285714285714
INFO:tensorflow:Average batch sequence length: 229.42857142857142
INFO:tensorflow:Average batch sequence length: 233.0
INFO:tensorflow:Average batch sequence length: 231.14285714285714
INFO:tensorflow:Average batch sequence length: 232.57142857142858
t5-large
INFO:tensorflow:Average batch sequence length: 229.57142857142858
INFO:tensorflow:Average batch sequence length: 236.0
INFO:tensorflow:Average batch sequence length: 240.0
INFO:tensorflow:Average batch sequence length: 231.85714285714286
INFO:tensorflow:Average batch sequence length: 230.71428571428572


In [12]:
t5_scores['t5-base']

{'precision': 25.18,
 'recall': 44.04,
 'f1': 30.99,
 'BLEURT': 0.5409001551866531,
 'BLEU': 46.305576394625874,
 'METEOR': 47.451796627600594,
 'rougeL': 29.570000876829624}

In [14]:
t5_scores_ef=computeAllScores(t5_ef)

t5-small
INFO:tensorflow:Average batch sequence length: 220.0
INFO:tensorflow:Average batch sequence length: 230.0
INFO:tensorflow:Average batch sequence length: 217.0
INFO:tensorflow:Average batch sequence length: 223.57142857142858
INFO:tensorflow:Average batch sequence length: 220.57142857142858
t5-base
INFO:tensorflow:Average batch sequence length: 238.0
INFO:tensorflow:Average batch sequence length: 236.57142857142858
INFO:tensorflow:Average batch sequence length: 237.57142857142858
INFO:tensorflow:Average batch sequence length: 239.71428571428572
INFO:tensorflow:Average batch sequence length: 234.28571428571428
t5-large
INFO:tensorflow:Average batch sequence length: 234.71428571428572
INFO:tensorflow:Average batch sequence length: 229.85714285714286
INFO:tensorflow:Average batch sequence length: 232.14285714285714
INFO:tensorflow:Average batch sequence length: 235.71428571428572
INFO:tensorflow:Average batch sequence length: 232.14285714285714


In [15]:
bart_baseline_scores=computeAllScores(bart_baseline)

bart-base
INFO:tensorflow:Average batch sequence length: 238.0
INFO:tensorflow:Average batch sequence length: 231.28571428571428
INFO:tensorflow:Average batch sequence length: 239.85714285714286
INFO:tensorflow:Average batch sequence length: 237.85714285714286
INFO:tensorflow:Average batch sequence length: 236.14285714285714
bart-large
INFO:tensorflow:Average batch sequence length: 235.14285714285714
INFO:tensorflow:Average batch sequence length: 219.85714285714286
INFO:tensorflow:Average batch sequence length: 225.42857142857142
INFO:tensorflow:Average batch sequence length: 238.42857142857142
INFO:tensorflow:Average batch sequence length: 236.57142857142858


In [16]:
bart_scores_ef=computeAllScores(bart_ef)

bart-base
INFO:tensorflow:Average batch sequence length: 241.0
INFO:tensorflow:Average batch sequence length: 238.28571428571428
INFO:tensorflow:Average batch sequence length: 235.28571428571428
INFO:tensorflow:Average batch sequence length: 251.28571428571428
INFO:tensorflow:Average batch sequence length: 233.42857142857142
bart-large
INFO:tensorflow:Average batch sequence length: 236.0
INFO:tensorflow:Average batch sequence length: 213.28571428571428
INFO:tensorflow:Average batch sequence length: 220.14285714285714
INFO:tensorflow:Average batch sequence length: 220.42857142857142
INFO:tensorflow:Average batch sequence length: 236.28571428571428


In [21]:
model_results ={'bart_ef':bart_scores_ef,
                'bart_baseline':bart_baseline_scores,
                't5_baseline':t5_scores,
                't5_ef':t5_scores_ef}

In [24]:
json.dump(model_results,open('../TrainedNarrators/model_results.json','w'))

In [22]:
model_results

{'bart_ef': {'bart-base': {'precision': 26.42,
   'recall': 48.83,
   'f1': 33.26,
   'BLEURT': 0.5526116153597832,
   'BLEU': 45.750068498712366,
   'METEOR': 47.82269593463433,
   'rouge1': 84.52439522091302,
   'rouge2': 59.273807072685926,
   'rougeL': 30.407036247518057},
  'bart-large': {'precision': 27.33,
   'recall': 46.64,
   'f1': 33.27,
   'BLEURT': 0.512464563548565,
   'BLEU': 46.97858229159241,
   'METEOR': 45.722011540832966,
   'rouge1': 85.210673549098,
   'rouge2': 58.38670351473212,
   'rougeL': 29.56711238083501}},
 'bart_baseline': {'bart-base': {'precision': 25.71,
   'recall': 49.26,
   'f1': 32.79,
   'BLEURT': 0.5471098693609238,
   'BLEU': 44.82653964203196,
   'METEOR': 47.82754921285556,
   'rouge1': 84.58404725577829,
   'rouge2': 58.19139514803832,
   'rougeL': 29.790561778986614},
  'bart-large': {'precision': 26.07,
   'recall': 46.74,
   'f1': 32.29,
   'BLEURT': 0.5129066711664201,
   'BLEU': 45.671373721083555,
   'METEOR': 46.413390102055864,
   'ro

In [17]:
t5_scores['t5-small']

{'precision': 26.71,
 'recall': 43.91,
 'f1': 32.12,
 'BLEURT': 0.5552313034534454,
 'BLEU': 37.83499039382401,
 'METEOR': 39.46466698502225,
 'rouge1': 75.4956327760751,
 'rouge2': 46.2877683219681,
 'rougeL': 31.874865817626947}

In [18]:
t5_scores_ef['t5-small']

{'precision': 27.7,
 'recall': 47.73,
 'f1': 34.04,
 'BLEURT': 0.5615042797327041,
 'BLEU': 40.57733052202082,
 'METEOR': 43.40056952532363,
 'rouge1': 78.9823504404105,
 'rouge2': 48.345355221414664,
 'rougeL': 31.974881006439226}

In [20]:
t5_scores['t5-base']

{'precision': 25.18,
 'recall': 44.04,
 'f1': 30.99,
 'BLEURT': 0.5409001551866531,
 'BLEU': 46.305576394625874,
 'METEOR': 47.451796627600594,
 'rouge1': 85.13247902208256,
 'rouge2': 60.02580878695297,
 'rougeL': 29.570000876829624}

In [19]:
t5_scores_ef['t5-base']

{'precision': 25.12,
 'recall': 45.73,
 'f1': 31.33,
 'BLEURT': 0.5446521639823914,
 'BLEU': 45.457930430656,
 'METEOR': 47.75205184684225,
 'rouge1': 84.57118976500631,
 'rouge2': 59.99163340146011,
 'rougeL': 29.78163320034269}

In [43]:
t5_scores_ef

{'t5-small': {'precision': 27.7,
  'recall': 47.73,
  'f1': 34.04,
  'BLEURT': 0.5615042797327041,
  'BLEU': 40.57733052202082,
  'METEOR': 43.40056952532363,
  'rougeL': 0.0},
 't5-base': {'precision': 25.12,
  'recall': 45.73,
  'f1': 31.33,
  'BLEURT': 0.5446521639823914,
  'BLEU': 45.457930430656,
  'METEOR': 47.75205184684225,
  'rougeL': 0.0},
 't5-large': {'precision': 25.35,
  'recall': 45.65,
  'f1': 31.58,
  'BLEURT': 0.5462669489383698,
  'BLEU': 46.0307056496799,
  'METEOR': 47.52666464749858,
  'rougeL': 0.0}}

In [98]:
import tempfile
from rouge_metric import PerlRouge
from rouge_metric import PyRouge
temp_dir=tempfile.TemporaryDirectory('exg')


In [100]:
rouge = PerlRouge(rouge_n_max=4, rouge_l=True, rouge_w=True,
    rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4,stemming=True)
def get_rouge(gen,ref):
    #score=metric_rouge.compute(predictions=[gen],references=ref,use_stemmer=True)
    #score= {key: value.high.fmeasure * 100 for key, value in score.items()}
    scores = rouge.evaluate(gen, trefs)
    return scores['rouge-l']['f']

In [107]:
bs= 8
ss=[]
for tt in t5_ef['t5-base']:
    gen= [normalize_text(t) for t in tt[f'{bs}']]
    
    #s = get_rouge(gen,orf)
    scores = get_rouge(gen,trefs)
    ss.append(scores)

In [108]:
np.mean(ss)

0.279396

In [94]:

scores = rouge.evaluate([normalize_text(t) for t in tt['8']], trefs)

In [90]:
rouge.evaluate??

In [96]:
scores['rouge-l']

{'r': 0.30862,
 'r_conf_int': (0.29218, 0.32722),
 'p': 0.30341,
 'p_conf_int': (0.28492, 0.32224),
 'f': 0.30118,
 'f_conf_int': (0.2858, 0.31747)}

In [50]:
ls /tmp/tmplbmiomzmexg/

gen1.1.txt  gen.1.txt  gen1.txt  gen.txt  ref.txt


In [46]:
ref_dir = os.path.join(temp_dir.name, 'gen1.1')
gg_dir = os.path.join(temp_dir.name, 'gen1')

In [47]:
writeToFile(refs[0],ref_dir)
writeToFile(tt['8'],gg_dir)

Done
Done


In [51]:
!rouge-metric /tmp/tmplbmiomzmexg/gen1.txt /tmp/tmplbmiomzmexg/gen1.1.txt -n 2 -w 1.2 -U -2 4

---------------------------------------------
A ROUGE-1 Average_R: 0.72665 (95%-conf.int. 0.72665 - 0.72665)
A ROUGE-1 Average_P: 0.71410 (95%-conf.int. 0.71410 - 0.71410)
A ROUGE-1 Average_F: 0.72032 (95%-conf.int. 0.72032 - 0.72032)
---------------------------------------------
A ROUGE-2 Average_R: 0.42936 (95%-conf.int. 0.42936 - 0.42936)
A ROUGE-2 Average_P: 0.42194 (95%-conf.int. 0.42194 - 0.42194)
A ROUGE-2 Average_F: 0.42562 (95%-conf.int. 0.42562 - 0.42562)
---------------------------------------------
A ROUGE-L Average_R: 0.70790 (95%-conf.int. 0.70790 - 0.70790)
A ROUGE-L Average_P: 0.69567 (95%-conf.int. 0.69567 - 0.69567)
A ROUGE-L Average_F: 0.70173 (95%-conf.int. 0.70173 - 0.70173)
---------------------------------------------
A ROUGE-W-1.2 Average_R: 0.10584 (95%-conf.int. 0.10584 - 0.10584)
A ROUGE-W-1.2 Average_P: 0.24866 (95%-conf.int. 0.24866 - 0.24866)
A ROUGE-W-1.2 Average_F: 0.14848 (95%-conf.int. 0.14848 - 0.14848)
---------------------------------------------
A 

In [109]:
trefs = [[normalize_text(r).split()] for r in refs[0]]

In [112]:
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
scores = rouge.evaluate_tokenized([normalize_text(t).split() for t in tt['8']], trefs)

In [113]:
scores['rouge-l']

{'r': 0.9173914047876188, 'p': 0.766393450167745, 'f': 0.8351218527714526}

In [61]:
scores['rouge-l']

{'r': 0.2977,
 'r_conf_int': (0.28131, 0.31582),
 'p': 0.29235,
 'p_conf_int': (0.27406, 0.3108),
 'f': 0.29032,
 'f_conf_int': (0.27503, 0.30652)}

In [35]:
!rouge-metric /tmp/tmplbmiomzmexg/gen.txt /tmp/tmplbmiomzmexg/ref.txt -n 2 -w 1.2 -U -2 4

Reference not found for /tmp/tmpyw_2wl4p/hyp/gen.txt


CalledProcessError: Command '['perl', '/home/essel/anaconda3/envs/annotation/lib/python3.8/site-packages/rouge_metric/RELEASE-1.5.5/ROUGE-1.5.5.pl', '-a', '-c', '95', '-e', '/home/essel/anaconda3/envs/annotation/lib/python3.8/site-packages/rouge_metric/RELEASE-1.5.5/data', '-n', '3', '-r', '1000', '-2', '4', '-U', '-w', '1.2', '-f', 'A', '-p', '0.5', './.rouge_metric/tmpkn78rv4e/config.xml']' returned non-zero exit status 255.

In [ ]:
ss

In [49]:
@background
def your_function(argument):
    time.sleep(5)
    print('function finished for '+str(argument))


for i in range(10):
    your_function(i)


print('loop finished')

loop finished
function finished for 0
function finished for 1
function finished for 4
function finished for 7
function finished for 6
function finished for 5
function finished for 9
function finished for 8
function finished for 2
function finished for 3
